In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# keep only docsuments with covid -cov-2 and cov2
def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    frames=[dfa,dfb,dfc,dfd]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

# load the meta data from the CSV file using some columns (abstract, title, authors),
df=pd.read_csv('metadata.csv', usecols=['title','journal','abstract','authors','doi','publish_time','sha'],engine='python',encoding='utf-8', error_bad_lines=False)

#drop duplicates
#df=df.drop_duplicates()
#drop NANs 
df=df.fillna('no data provided')
df = df.drop_duplicates(subset='title', keep="first")
df=df[df['publish_time'].str.contains('2020')]
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()
#show 5 lines of the new dataframe
df=search_focus(df)
print (df.shape)
df.head()

Skipping line 139952: unexpected end of data


(22982, 7)


,sha,title,doi,abstract,publish_time,authors,journal
4534,no data provided,Latest updates on COVID-19 from the European C...,10.2807/1560-7917.es.2020.25.6.2002131,no data providedlatest updates on covid-19 fro...,2020-02-13,no data provided,Euro Surveill
4622,no data provided,Latest assessment on COVID-19 from the Europea...,10.2807/1560-7917.es.2020.25.8.2002271,no data providedlatest assessment on covid-19 ...,2020-02-27,no data provided,Euro Surveill
4655,no data provided,Updated rapid risk assessment from ECDC on the...,10.2807/1560-7917.es.2020.25.9.2003051,no data providedupdated rapid risk assessment ...,2020-03-05,no data provided,Euro Surveill
4688,no data provided,Updated rapid risk assessment from ECDC on the...,10.2807/1560-7917.es.2020.25.10.2003121,no data providedupdated rapid risk assessment ...,2020-03-12,no data provided,Euro Surveill
7599,43c92bb293ea5ea1be4c5a6da11fe517b2b5677e,COVID-19 and gender-specific difference: Analy...,10.1017/ice.2020.64,no data providedcovid-19 and gender-specific d...,2020-03-09,"Zhao, Shi; Cao, Peihua; Chong, Marc K.C.; Gao,...",no data provided


In [ ]:
import functools
from IPython.core.display import display, HTML
from nltk import PorterStemmer

# function to stem keyword list into a common base word
def stem_words(words):
    stemmer = PorterStemmer()
    singles=[]
    for w in words:
        singles.append(stemmer.stem(w))
    return singles

def search_dataframe(df,search_words):
    search_words=stem_words(search_words)
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search_words))]
    return df1

# function analyze search results for relevance with word count / abstract length
def search_relevance(rel_df,search_words):
    rel_df['score']=""
    search_words=stem_words(search_words)
    for index, row in rel_df.iterrows():
        abstract = row['abstract']
        result = abstract.split()
        len_abstract=len(result)
        score=0
        for word in search_words:
            score=score+result.count(word)
        final_score=(score/len_abstract)
        rel_score=score*final_score
        rel_df.loc[index, 'score'] = rel_score
    rel_df=rel_df.sort_values(by=['score'], ascending=False)
    #rel_df= rel_df[rel_df['score'] > .01]
    return rel_df

# function to get best sentences from the search results
def get_sentences(df1,search_words):
    df_table = pd.DataFrame(columns = ["pub_date","authors","title","excerpt","rel_score"])
    search_words=stem_words(search_words)
    for index, row in df1.iterrows():
        pub_sentence=''
        sentences_used=0
        #break apart the absracrt to sentence level
        sentences = row['abstract'].split('. ')
        #loop through the sentences of the abstract
        highligts=[]
        for sentence in sentences:
            # missing lets the system know if all the words are in the sentence
            missing=0
            #loop through the words of sentence
            for word in search_words:
                #if keyword missing change missing variable
                if word not in sentence:
                    missing=1
                #if '%' in sentence:
                    #missing=missing-1
            # after all sentences processed show the sentences not missing keywords
            if missing==0 and len(sentence)<1000 and sentence!='':
                sentence=sentence.capitalize()
                if sentence[len(sentence)-1]!='.':
                    sentence=sentence+'.'
                pub_sentence=pub_sentence+'<br><br>'+sentence
        if pub_sentence!='':
            sentence=pub_sentence
            sentences_used=sentences_used+1
            authors=row["authors"].split(" ")
            link=row['doi']
            title=row["title"]
            score=row["score"]
            linka='https://doi.org/'+link
            linkb=title
            sentence='<p fontsize=tiny" align="left">'+sentence+'</p>'
            final_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,linkb)
            to_append = [row['publish_time'],authors[0]+' et al.',final_link,sentence,score]
            df_length = len(df_table)
            df_table.loc[df_length] = to_append
    return df_table
    
################ main program ########################

#display(HTML('<h1>Task 1: What is known about transmission, incubation, and environmental stability?</h1>'))

display(HTML('<h3>Results currently limited to two (10) for ease of scanning</h3>'))

# list of lists of search terms
questions=[
['1: What is the range of incubation periods for the disease in humans?'],
['2: How long are individuals are contagious?'],
['3: Even after recovery, how long are individuals are contagious?.'],
['4: Does the range of incubation period vary across age groups?'],
['5: Does the range of incubation period vary with children?'],
['6: Does the range of incubation period vary based on underlying health?'],
['7: What do we know about the basic reproduction number?'],
['8: What is the prevalance of asymptomatic transmission?'],
['9: What do we know about asymptomatic transmission in children?'],
['10: What do we know about seasonality of transmission?'],
['11: Informing decontamination based on physical science of the coronavirus?'],
['12: What do we know about stability of the virus in environmental conditions?'],
['13: What do we know about persistence of the virus on various substrates? (e.g., nasal discharge, sputum, urine, fecal matter, blood)'],
['14: What do we know about persistence of the virus on various surfaces? (e,g., copper, stainless steel, plastic) '],
['15: What do we know about viral shedding duration?'],
['16: What do we know about viral shedding in fecal/stool?'],
['17: What do we know about viral shedding from nasopharynx?'],
['18: What do we know about viral shedding in blood?'],
['19: What do we know about viral shedding in urine?'],
['20: What do we know about implemtation of diagnostics?'],
['21: What do we know about disease models?'],
['22: What do we know about models for disease infection?'],
['23: What do we know about models for disease transmission?'],
['24: Are there studies about phenotypic change?'],
['25: What is know about adaptations (mutations) of the virus?'],
['26: What do we know about the human immune response and immunity?'],
['27: Is population movement control effective in stopping transmission (spread)?'],
['28: Effectiveness of personal protective equipment (PPE)?'],
['29: What is the role of environment in transmission?']
]
#search=[['incubation','period','range','mean','%']]
search=[['incubation','period','range'],
['viral','shedding','duration'],
['asymptomatic','shedding'],
['incubation','period','age','statistically','significant'],
['incubation','period','child'],
['incubation','groups','risk'],
['basic','reproduction', 'number','%'],
['asymptomatic','infection','%'],
['asymptomatic','children'],
['seasonal','transmission'],
['contaminat','object'],
['environmental', 'conditions'],
['sputum','stool','blood','urine'],
['persistence','surfaces'],
['duration','viral','shedding'],
['shedding','stool'],
['shedding','nasopharynx'],
['shedding','blood'],
['shedding','urine'],
['diagnostics','point'],
['model', 'disease'],        
['model', 'infection'],
['model', 'transmission'],
['phenotypic'],
['mutation'],
['immune','response'],
['restriction', 'movement'],
['protective','clothing'],
['transmission','routes']
]
q=0
for search_words in search:
    str1=''
    # a make a string of the search words to print readable version above table
    str1=' '.join(questions[q])
    
    #search the dataframe for all words
    df1=search_dataframe(df,search_words)

    # analyze search results for relevance 
    df1=search_relevance(df1,search_words)

    # get best sentences
    df_table=get_sentences(df1,search_words)
    
    length=df_table.shape[0]
    #limit 3 results
    df_table=df_table.head(15)
    df_table=df_table.drop(['rel_score'], axis=1)
    #convert df to html
    df_table=HTML(df_table.to_html(escape=False,index=False))
    
    # display search topic
    display(HTML('<h3>'+str1+'</h3>'))
    
    #display table
    if length<1:
        print ("No reliable answer could be located in the literature")
    else:
        display(df_table)
    q=q+1
print ('done')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pub_date,authors,title,excerpt
2020-03-08,"Yang, et al.",[The preliminary analysis on the characteristics of the cluster for the Corona Virus Disease].,"We selected 325 cases to estimate the incubation period and found its range is 1 to 20 days, median was 7 days, and mode was 4 days."
2020-12-01,"Chen, et al.",Epidemiological analysis of 18 patients with COVID-19.,The epidemiological characteristics were as follows: (1) the median incubation period was 8 days (with an interquartile range of 4-12 days); (2) the incubation period in one case was ≥18 days; (3) one infant patient was asymptomatic prior to their diagnosis; and (4) two asymptomatic patients had a positive nucleic acid test after their family members were diagnosed with covid-19.
2020-08-07,"Liu, et al.",Analysis of community-acquired COVID-19 cases in Taiwan.,The median incubation period was 6 days (range 1 to 13 days) and the median serial interval was 4 days (range -3 to 24 days).
2020-09-03,"Akimkin, et al.",[Patterns of the SARS-CoV-2 epidemic spread in a megacity].,"It was discovered that the detection frequency of sars-cov-2 rna in relatively healthy population and its dynamics are important monitoring parameters, especially during the increase and stabilization in the covid-19 incidence, and are instrumental in predicting the development of the epidemic situation within a range of 1-2 incubation periods (14-28 days)."
2020-06-01,"Paglia, et al.",COVID-19 and Paediatric Dentistry after the lockdown.,"The average incubation period is about 5 days, with an estimated range from 2 to 14 days; the incubation period in children is similar, however some have exhibited a longer incubation."
2020-05-14,"Madabhavi, et al.",COVID-19: a review.,"A highly virulent and pathogenic covid-19 viral infection with incubation period ranging from two to fourteen days, transmitted by breathing of infected droplets or contact with infected droplets, belongs to the genus coronavirus with its high mutation rate in the coronaviridae."
2020-12-03,"Dërmaku-Sopjani, et al.",Molecular characterization of SARS-CoV-2.,The incubation period ranges from 1 to 14 days.
2020-04-06,"Han, et al.",A comparative-descriptive analysis of clinical characteristics in 2019-Coronavirus-infected children and adults.,"The median incubation period of children and adults was 5 days (range 3-12 days) and 4 days (range 2-12 days), respectively."
2020-12-22,Al et al.,COVID-19: An Update on Pathogenesis and Treatment.,Symptoms start to appear after the incubation period of the virus which ranges from 2 to 14 days.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pub_date,authors,title,excerpt
2020-12-01,"Pan, et al.",Retrospective analysis of the effect of current clinical medications and clinicopathological factors on viral shedding in COVID-19 patients.,"Detailed clinical data of each patient were collected, and the factors that affected the duration of viral shedding were retrospectively analysed.The median duration of viral shedding in the 186 covid-19 patients was 13 days.The median duration of viral shedding was 12 days in non-severe patients, and 17 days in severe patients, and there was a significant difference between the two groups (p<0.001).Spearman's rank correlation analysis showed that the onset-drug interval was positively correlated with the duration of viral shedding (r=0.446; p<0.0001).Lpv/r shortened the duration of viral shedding, and the smaller the interval between presentation and lpv/r onset was, the faster viral shedding occurred.retrospective analysis of the effect of current clinical medications and clinicopathological factors on viral shedding in covid-19 patients."
2020-07-21,"Dodds, et al.",Model-Informed Drug Repurposing: Viral Kinetic Modeling to Prioritize Rational Drug Combinations for COVID-19.,"The endpoints and metrics included viral load area under the curve (auc), duration of viral shedding, and epithelial cells infected.In addition, we observed that the time-window opportunity for a therapeutic intervention to effect duration of viral shedding exceeds the effect on sparing epithelial cells from infection or impact on viral load auc."
2020-12-31,"Nam, et al.",540. Prolonged Viral Shedding of SARS-CoV-2 In Solid Organ Transplant Recipients,"The duration of viral shedding in sot recipients with sars-cov-2 infection is unknown.Viral shedding duration was obtained by evaluating all subsequent sars-cov-2 pcr results following initial positivity over time.Prolonged viral shedding was observed in few patients, with median duration of sars-cov-2 pcr positivity at 32 (iqr 18.5 – 41.0) days.Demographics and outcomes [image: see text] duration of viral shedding in sot patients with covid-19 [image: see text] conclusion: covid-19 can lead to significant outcomes in sot with increased mortality in those with severe disease, as well as prolonged viral shedding.Further studies are needed to elucidate the full duration of viral shedding in this population."
2020-06-30,"Xu, et al.",Duration of Respiratory and Gastrointestinal Viral Shedding in Children With SARS-CoV-2: A Systematic Review and Synthesis of Data.,"Little is known about the duration of respiratory and gastrointestinal viral shedding in children with covid-19.Methods we performed a systematic search of ovid medline, embase and cochrane central databases for studies reporting real-time reverse transcriptase polymerase chain reaction (rt-pcr) results in children with covid-19, then extracted and synthesized data on duration of viral shedding from symptom onset in respiratory and gastrointestinal samples.Results based on data compiled from 69 pediatric cases, the duration of viral shedding through the respiratory tract is up to 24 days from symptom onset with a mean of 11.1 ± 5.8 days.The mean duration of viral shedding via the gastrointestinal tract was 23.6 ± 8.8 days from symptom onset.Conclusions to our knowledge, this is the first attempt to systematically review the duration of respiratory and gastrointestinal viral shedding of sars-cov-2 in pediatric patients.These findings may have important implications for infection control strategies during the covid-19 pandemic.duration of respiratory and gastrointestinal viral shedding in children with sars-cov-2: a systematic review and synthesis of data."
2020-04-10,"Qian, et al.",Duration of SARS-CoV-2 viral shedding during COVID-19 infection.,No data providedduration of sars-cov-2 viral shedding during covid-19 infection.
2020-04-03,"Chen, et al.",The Presence of SARS-CoV-2 RNA in Feces of COVID-19 Patients.,"The duration of viral shedding from feces af

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pub_date,authors,title,excerpt
2020-07-17,"Fisher, et al.","Factors Associated with Cloth Face Covering Use Among Adults During the COVID-19 Pandemic - United States, April and May 2020.","The use of a cloth face covering was associated with theory-derived constructs that indicate a favorable attitude toward them, intention to use them, ability to use them, social support for using them, and beliefs that they offered protection for self, others, and the community."
2020,"Silva, et al.",Cloth masks as respiratory protections in the COVID-19 pandemic period: evidence gaps.,"Objective to identify scientific evidence on the effectiveness of using cloth masks as safe protectors against covid-19.Evidence shows that cloth masks do not have the same protective characteristics as surgical masks, indicating an increased risk of infection due to humidity, diffusion of fluids, virus retention, and improper preparation.Conclusion cloth masks should be used together with preventive measures, such as home insulation, good respiratory conduct, and regular hand hygiene.cloth masks as respiratory protections in the covid-19 pandemic period: evidence gaps."
2020-12-10,"Clapp, et al.",Evaluation of Cloth Masks and Modified Procedure Masks as Personal Protective Equipment for the Public During the COVID-19 Pandemic.,"Conclusions and relevance while modifications to improve medical procedure mask fit can enhance the filtering capability and reduce inhalation of airborne particles, this study demonstrates that the ffes of consumer-grade masks available to the public are, in many cases, nearly equivalent to or better than their non-n95 respirator medical mask counterparts.evaluation of cloth masks and modified procedure masks as personal protective equipment for the public during the covid-19 pandemic."
2020-10-22,"Tang, et al.",Daylight-Induced Antibacterial and Antiviral Cotton Cloth for Offensive Personal Protection.,"This work demonstrated that the cationized cotton could serve as a platform for different functionalization applications, and the resulting fiber materials could inspire the development of reusable and sustainable ppe with significant bioprotective properties to fight the covid-19 pandemic as well as the spread of other contagious diseases.daylight-induced antibacterial and antiviral cotton cloth for offensive personal protection."
2020-04-01,"Antal, et al.",[New information for the clinical detection of COVID-19 virus infection and options for protection of healthcare workers in the head and neck region].,"Based on the processed publications, it can be stated that during the interventions primarily affecting the head and neck region, especially in the case of interventions requiring the use of a rotating device, it is essential to fully protect all staff in the room with disposable full clothing, goggles and special masks."
2020,"Solia, et al.",The Role of Surgical Masks during the COVID-19 Pandemic. A Mini-Review.,"However, surgical masks primarily and cloth masks secondarily seem to be adequate means of protection for the general public and for healthcare workers during procedures where respiratory droplets are not generated."
2020-02-17,"Wang, et al.",Challenges to the system of reserve medical supplies for public health emergencies: reflections on the outbreak of the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) epidemic in China.,"As the pandemic has developed, a fact has become apparent: there is a serious dearth of emergency medical supplies, and especially an extreme shortage of personal protective equipment such as masks and medical protective clothing."
2020,"Jonckheere, et al.",Percutaneous tracheostomy for long-term ventilated COVID-19-patients: rationale and first clinical-safe for all-experience.,"Material and methods we present a guidewire dilating forceps tracheostomy procedure in covid-19 patients that was optimised including apnoea-moments, protective clothing, checklists, and clear protocols."
2020-

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pub_date,authors,title,excerpt
2020,"Miranda, et al.",Head and neck cancer therapy-related oral manifestation management in the COVID-19 pandemic: a critical review.,"Because dental professionals are in close contact with the main viral transmission routes, this study presents recommendations for management and protection during clinical dental care.The main characteristics and transmission routes of covid-19 are also discussed."
2020-12-04,"Gola, et al.","SARS-CoV-2 indoor contamination: considerations on anti-COVID-19 management of ventilation systems, and finishing materials in healthcare facilities.","Starting from the sars-cov-2 transmission routes, the paper investigates the hospital risk analysis and management, the indoor air quality and determination of microbial load, surface management and strategies in cleaning activities, hvac systems' management and filters' efficiency."
2020-04-14,"Zhang, et al.",[Whether conjunctival congestion in patients infected with 2019 novel coronavirus is definitely related the novel coronavirus?],"Currently, it is commonly reported that covid-19 patients are associated with conjunctival congestion and other symptoms, and the transmission route or high-risk diseases of ophthalmology and high-risk working status are speculated."
2020-09-01,"Ralli, et al.","Impact of COVID-19 pandemic on otolaryngology, ophthalmology and dental clinical activity and future perspectives.","Objective since the covid-19 outbreak, otolaryngologists, ophthalmologists and dentists have been severely affected, both for the transmission routes and for the diagnostical and therapeutic procedures typical of these disciplines.Materials and methods in this article, we discuss the transmission routes, the potential risk of contagion for patients and healthcare providers during procedures, and comment on the changes that will affect head and neck clinical practice in the future of outpatient and surgical activities."
2020-06-19,"Peres, et al.",[Respiratory and Facial Protection: Current Perspectives in the Context of the COVID-19 Pandemic].,"This equipment must be appropriate to the pathogen transmission route that, in the case of sars-cov-2, occurs through droplet and contact routes.The use of different, or higher than necessary, level of personal protective equipment, for the transmission route of the agent, is a form of misuse and can affect its supply for situations when it is clearly indicated."
2020,"Yilmaz, et al.","The Assessment of Knowledge, Behaviors, and Anxiety Levels of the Orthodontists about COVID-19 Pandemic.","Results most of the orthodontists were aware of covid-19 symptoms and transmission routes (n=159 and n=183, respectively)."
2020-09-30,"Kaya, et al.",Detection of SARS-CoV-2 in the tears and conjunctival secretions of Coronavirus disease 2019 patients.,Introduction current studies suggest that tears and conjunctival secretions may be an important transmission route in coronavirus disease 2019 (covid-19).
2020-11-01,"Vatan, et al.",Knowledge and attitudes towards COVID-19 among emergency medical service workers.,"Overall, > 96% of the participants had adequate knowledge about the transmission routes of covid-19."
2020-06-01,"Paglia, et al.",COVID-19 and Paediatric Dentistry after the lockdown.,"We know that the route of transmission of the disease is through saliva droplets produced by speaking, coughing and sneezing."
2020-06-28,"Li, et al.","Knowledge, attitude, and practice survey regarding coronavirus disease 2019 among residents in Hunan Province.","Residents had the highest score of major symptoms of covid-19 (3.96±0.39), but the lowest was the main transmission routes (3.47±0.89)."


done
